<a href="https://colab.research.google.com/github/mhoussam1/Group-1-COVID19/blob/Iffadanwar/ML_start.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [80.4 kB]
Hit:12 http://ppa.launchpad.net/cran/

In [4]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-03-21 03:48:42--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.64MB/s    in 0.2s    

2022-03-21 03:48:42 (5.64 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [67]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [68]:
import findspark
findspark.init()
import requests
import json
import sys

from pyspark.sql.functions import *
from pyspark.sql import functions as sf
from pyspark.sql.types import StructType, StructField, IntegralType, StringType, ArrayType
from pyspark.sql import SparkSession
from pyspark import *
from pyspark.context import SparkContext

import pandas as pd
import pyspark
from pathlib import Path
from collections import Counter

from urllib.request import Request,urlopen

In [69]:
import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [70]:
from pyspark.sql.types import StructType,StructField, StringType
from pyspark.sql.functions import col,from_json


In [71]:
http = urlopen("https://api.covid19api.com/summary").read().decode("utf-8")

In [72]:
df2 = spark.sparkContext.parallelize([http])

covid_data = spark.read.option("multiline","true").json(df2)
covid_data.show(10)

+--------------------+--------------------+--------------------+--------------------+-------+
|           Countries|                Date|              Global|                  ID|Message|
+--------------------+--------------------+--------------------+--------------------+-------+
|[[Afghanistan, AF...|2022-03-21T03:07:...|[2022-03-21T03:07...|56a9b298-64f0-4f9...|       |
+--------------------+--------------------+--------------------+--------------------+-------+



In [73]:
covid_data.printSchema()

root
 |-- Countries: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- Country: string (nullable = true)
 |    |    |-- CountryCode: string (nullable = true)
 |    |    |-- Date: string (nullable = true)
 |    |    |-- ID: string (nullable = true)
 |    |    |-- NewConfirmed: long (nullable = true)
 |    |    |-- NewDeaths: long (nullable = true)
 |    |    |-- NewRecovered: long (nullable = true)
 |    |    |-- Slug: string (nullable = true)
 |    |    |-- TotalConfirmed: long (nullable = true)
 |    |    |-- TotalDeaths: long (nullable = true)
 |    |    |-- TotalRecovered: long (nullable = true)
 |-- Date: string (nullable = true)
 |-- Global: struct (nullable = true)
 |    |-- Date: string (nullable = true)
 |    |-- NewConfirmed: long (nullable = true)
 |    |-- NewDeaths: long (nullable = true)
 |    |-- NewRecovered: long (nullable = true)
 |    |-- TotalConfirmed: long (nullable = true)
 |    |-- TotalDeaths: long (nullable = true)
 |    |--

In [74]:
df_explode = covid_data.withColumn("Countries", sf.explode(sf.col("Countries"))).select(["Countries.Country","Countries.CountryCode","Countries.Date","Countries.ID","Countries.NewConfirmed","Countries.NewDeaths","Countries.NewRecovered","Countries.TotalConfirmed","Countries.TotalDeaths","Countries.TotalRecovered",])
df_explode.head()

Row(Country='Afghanistan', CountryCode='AF', Date='2022-03-21T03:07:57.329Z', ID='c146b0b7-6c72-440c-8e55-1617a479138f', NewConfirmed=54, NewDeaths=2, NewRecovered=0, TotalConfirmed=177093, TotalDeaths=7654, TotalRecovered=0)

In [75]:
schema = df_explode.selectExpr( 
                                "cast(Country as string) Country",
                                "cast(CountryCode as string) CountryCode",
                                "cast(Date as date) Date",
                                "cast(ID as string) ID",
                                "cast(NewConfirmed as int) NewConfirmed",
                                "cast(NewDeaths as int) NewDeaths",
                                "cast(NewRecovered as int) NewRecovered",
                                "cast(TotalConfirmed as int) TotalConfirmed",
                                "cast(TotalDeaths as int) TotalDeaths",
                                "cast(TotalRecovered as int) TotalRecovered"
  )
schema.printSchema()

root
 |-- Country: string (nullable = true)
 |-- CountryCode: string (nullable = true)
 |-- Date: date (nullable = true)
 |-- ID: string (nullable = true)
 |-- NewConfirmed: integer (nullable = true)
 |-- NewDeaths: integer (nullable = true)
 |-- NewRecovered: integer (nullable = true)
 |-- TotalConfirmed: integer (nullable = true)
 |-- TotalDeaths: integer (nullable = true)
 |-- TotalRecovered: integer (nullable = true)



In [76]:
covid_df = schema.toPandas()
covid_df.head()

,Country,CountryCode,Date,ID,NewConfirmed,NewDeaths,NewRecovered,TotalConfirmed,TotalDeaths,TotalRecovered
0,Afghanistan,AF,2022-03-21,c146b0b7-6c72-440c-8e55-1617a479138f,54,2,0,177093,7654,0
1,Albania,AL,2022-03-21,f3942d0b-40a8-433d-98bd-9f2f36fb2eb4,48,0,0,273088,3487,0
2,Algeria,DZ,2022-03-21,4af42d1d-31cd-4f83-9f8d-3005d2b0b917,11,0,0,265550,6871,0
3,Andorra,AD,2022-03-21,180a07fc-cb91-459f-bcf5-93ddde4e40b1,0,0,0,39234,153,0
4,Angola,AO,2022-03-21,105876a2-27fd-43f0-a828-69f3a57ba6dc,0,0,0,99003,1900,0


In [77]:
covid_df

,Country,CountryCode,Date,ID,NewConfirmed,NewDeaths,NewRecovered,TotalConfirmed,TotalDeaths,TotalRecovered
0,Afghanistan,AF,2022-03-21,c146b0b7-6c72-440c-8e55-1617a479138f,54,2,0,177093,7654,0
1,Albania,AL,2022-03-21,f3942d0b-40a8-433d-98bd-9f2f36fb2eb4,48,0,0,273088,3487,0
2,Algeria,DZ,2022-03-21,4af42d1d-31cd-4f83-9f8d-3005d2b0b917,11,0,0,265550,6871,0
3,Andorra,AD,2022-03-21,180a07fc-cb91-459f-bcf5-93ddde4e40b1,0,0,0,39234,153,0
4,Angola,AO,2022-03-21,105876a2-27fd-43f0-a828-69f3a57ba6dc,0,0,0,99003,1900,0
...,...,...,...,...,...,...,...,...,...,...
189,Venezuela (Bolivarian Republic),VE,2022-03-21,9a5be261-a821-4a9b-8397-dc5047e887fd,138,1,0,519558,5667,0
190,Viet Nam,VN,2022-03-21,8c1f0087-7f15-4053-80c7-b026253cd570,166207,63,0,7958048,41880,0
191,Yemen,YE,2022-03-21,2c2f5375-ed2d-41e5-a553-b1f0cfc26eb2,2,0,0,11801,2140,0
192,Zambia,ZM,2022-03-21,9b0a23d9-e912-49fa-bffa-264464711896,0,0,0,315623,3962,0


In [78]:
mean = covid_df.mean()
mean

NewConfirmed      4.169005e+03
NewDeaths         1.642784e+01
NewRecovered      0.000000e+00
TotalConfirmed    2.419590e+06
TotalDeaths       3.129379e+04
TotalRecovered    0.000000e+00
dtype: float64

In [83]:
covid_df = covid_df.drop(["NewRecovered" ,"TotalRecovered"], axis=1)

In [85]:
X = covid_df.drop(["TotalDeaths", "Country", "Date", "ID", "CountryCode"], axis=1)
y = covid_df["TotalDeaths"]
X.head()


,NewConfirmed,NewDeaths,TotalConfirmed
0,54,2,177093
1,48,0,273088
2,11,0,265550
3,0,0,39234
4,0,0,99003


In [86]:
y.value_counts()

0        5
365      2
163      2
125      2
189      2
        ..
217      1
17231    1
440      1
169      1
5426     1
Name: TotalDeaths, Length: 186, dtype: int64

In [87]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

ValueError: ignored

In [88]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
# Instantiate
brfc = BalancedRandomForestClassifier(n_estimators=100, random_state=1)

# Fit
brfc.fit(X_train, y_train)

NameError: ignored

In [89]:
# Calculated the balanced accuracy score
y_pred = brfc.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

NameError: ignored

In [90]:
0.7877672625306695
# Display the confusion matrix
# Calculating the confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual high_risk", "Actual low_risk"], columns=["Predicted high_risk", "Predicted low_risk"])
cm_df

NameError: ignored